In [ ]:
from sentence_transformers import models, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader

In [ ]:
sts_reader = STSDataReader(dataset_folder="./stsvv-clean")

test_set = sts_reader.get_examples("test.csv")

In [ ]:
model = models.Transformer("./model.zip")

model.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.auto_model.resize_token_embeddings(len(model.tokenizer))

pooling_model = models.Pooling(model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[model, pooling_model])

In [ ]:
from sentence_transformers import SentenceTransformer, util

predictions = []
for example in test_set:
    sentence1 = example.texts[0]
    sentence2 = example.texts[1]
    label = example.label

    sentence_embeddings1 = model.encode(sentence1, show_progress_bar=False)
    sentence_embeddings2 = model.encode(sentence2, show_progress_bar=False)

    cos_similarity = util.cos_sim(sentence_embeddings1, sentence_embeddings2)
    
    predictions.append((sentence1, sentence2, label, round(float(cos_similarity), 2)))

incorrect_predictions = []
for example in predictions:
    sentence1, sentence2, label, prediction = example
    
    temp_pred = 1 if prediction >= 0.5 else 0
    temp_label = 1 if label >= 0.5 else 0
    
    if temp_pred != temp_label:
        incorrect_predictions.append(example)

print((len(predictions) - len(incorrect_predictions))/len(predictions))

In [ ]:
sentence_1 = model.encode("Ngày mai quân lính sẽ đến dọn dẹp tàn dư của cuộc chiến.", show_progress_bar=False)
sentence_2 = model.encode("Một cuộc dọn dẹp sẽ được thực hiện vào ngày mai bởi quân đội.", show_progress_bar=False)

cos_similarity = util.cos_sim(sentence_1, sentence_2)

print("Similar" if round(float(cos_similarity)) >= 0.5 else "Not similar")